In [1]:
import numpy as np

# numerical_flux.py

(2, 3, 3)
[[[1 1]
  [2 2]
  [1 1]]

 [[2 2]
  [4 4]
  [2 2]]

 [[1 1]
  [2 2]
  [1 1]]]
(2, 3, 3)
[[[1 2 1]
  [2 4 2]
  [1 2 1]]

 [[1 2 1]
  [2 4 2]
  [1 2 1]]]


In [181]:
A = np.arange(25).reshape(5,5)
A = A[...]
B = np.stack((A,A,A), axis=0)
print(B.shape)
# print(B[:,:,0])
# B = np.transpose(B, (0,1,2))

axes = [1,0,2]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

# axes = [2,1,0]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

B = np.moveaxis(B, 1, 2)
# print(B)

B = np.moveaxis(B, 1, 2)
# print(B)
# print(B.ndim)
# B = np.moveaxis(B,0,-1)
print(B)

# print(B.reshape(-1))

cnt, cntx, cnty, cntz = 0, 0, 0, 0
nsnc = B.reshape(-1).shape[0]

iczn, icyn, icxn = B.shape

for idx in range(nsnc):
    
#     print("cnt=%i, cntx=%i, cnty=%i, cntz=%i, cnty_r=%i" %(cnt, cntx, cnty, cntz, cnty % (icxn * icyn)))
    
    cntx += 1
    if cntx % icxn == 0:
        cnty += 1
        cntx = 0
        
        if cnty % icyn == 0:
            cntz += 1
            cnty = 0
            
    cnt += 1

(3, 5, 5)
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]]


In [300]:
import numba as nb
from scipy import signal

@nb.jit(nopython=True, cache=True)
def dict_func(lst,a):
    plst = lst[1:-1,1:-1,:]
    leftx = plst[:,:,:-1]
    rightx = plst[:,:,1:]

    x_fluxes = (rightx - leftx)
#     print(x_fluxes.shape)

    plst = lst[1:-1,:,1:-1]
    lefty = plst[:,:-1,:]
    righty = plst[:,1:,:]

    y_fluxes = (righty - lefty)
#     print(y_fluxes.shape)

    plst = lst[:,1:-1,1:-1]
    leftz = plst[:-1,:,:]
    rightz = plst[1:,:,:]

    z_fluxes = (rightz - leftz)
#     print(z_fluxes.shape)
    lst = - x_fluxes[:,:,:-1] + x_fluxes[:,:,1:] \
        - y_fluxes[:,:-1,:] + y_fluxes[:,1:,:] - z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]
#     lst = lst / 16
    return lst
    
# np.random.seed(555)
print(lst.shape)
shp = lst.shape

# # p = np.zeros_like(lst)
# plst = lst.tolist()
# plst = lst[:,:,-1][...,None] + lst[:,:,:] + lst[:,:,0][...,None]
lst = np.pad(lst,((0,0),(0,0),(1,1)),mode='wrap')
lst = np.pad(lst,((0,0),(1,1),(0,0)),mode='wrap')
lst = np.pad(lst,((1,1),(0,0),(0,0)),mode='constant')

# plst = lst[1:-1,1:-1,:]
# leftx = plst[:,:,:-1]
# rightx = plst[:,:,1:]

# x_fluxes = (rightx - leftx)
# print(x_fluxes.shape)

# plst = lst[1:-1,:,1:-1]
# lefty = plst[:,:-1,:]
# righty = plst[:,1:,:]

# y_fluxes = (righty - lefty)
# print(y_fluxes.shape)

# plst = lst[:,1:-1,1:-1]
# leftz = plst[:-1,:,:]
# rightz = plst[1:,:,:]

# z_fluxes = (rightz - leftz)
# print(z_fluxes.shape)

# p = - x_fluxes[:,:,:-1] + x_fluxes[:,:,1:] \
#     - y_fluxes[:,:-1,:] + y_fluxes[:,1:,:] - z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]

# p = p * 1. / 16
# print(p.shape)

a = np.ones(lst.shape)
for i in range(200):
    lst = np.random.randint(15,size=(64,35,64))
    dict_func(lst,a)
#     print(lst)
# print(lst_massaged)

(64, 35, 64)
6
14
7
10
3
0
7
6
3
6
14
8
9
11
3
13
14
9
10
1
6
3
11
14
2
7
12
14
12
12
13
6
12
7
10
12
7
4
7
5
7
14
9
9
1
1
13
11
13
3
6
1
10
2
9
10
14
5
8
1
7
5
3
9
0
9
1
7
12
9
7
11
6
8
5
1
8
2
4
10
11
14
11
2
9
7
0
13
12
1
4
11
1
12
3
8
0
0
8
4
5
7
2
12
12
6
2
0
0
7
6
2
1
6
12
14
12
1
14
5
13
8
3
6
2
13
1
2
11
14
2
8
6
7
0
4
3
3
7
10
10
10
1
3
10
1
4
13
11
14
13
1
4
5
5
2
11
9
6
14
13
7
9
12
9
10
7
5
9
1
10
9
6
3
12
7
5
8
2
10
10
9
2
3
5
11
9
13
4
11
2
2
2
7
11
11
6
10
13
9
